In [1]:
import tensorflow as tf
import os
import numpy as np
from glob import glob as glb
from tqdm import *
import h5py
from tensorflow.contrib import rnn

min_queue_examples = 1000

/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p


In [2]:
def load_flow(filename, shape):
  stream_flow = h5py.File(filename, 'r')
  flow_state_vel = np.array(stream_flow['Velocity_0'][:])
  flow_state_vel = flow_state_vel.reshape([shape[0], shape[1]+128, 3])[0:shape[0],0:shape[1],0:2]
  stream_flow.close()
  return flow_state_vel

def load_boundary(filename, shape):
  stream_boundary = h5py.File(filename, 'r')
  boundary_cond = np.array(stream_boundary['Gamma'][:])
  boundary_cond = boundary_cond.reshape([shape[0], shape[1]+128, 1])[0:shape[0],0:shape[1],:]
  stream_boundary.close()
  return boundary_cond

def read_data(filename_queue, shape):
  reader = tf.TFRecordReader()
  key, serialized_example = reader.read(filename_queue)
  features = tf.parse_single_example(
    serialized_example,
    features={
      'boundary':tf.FixedLenFeature([],tf.string),
      'sflow':tf.FixedLenFeature([],tf.string),
      'vmax':tf.FixedLenFeature([],tf.string),
    }) 
  boundary = tf.decode_raw(features['boundary'], tf.uint8)
  sflow = tf.decode_raw(features['sflow'], tf.float32)
  vmax = tf.decode_raw(features['vmax'], tf.float32)

  boundary = tf.reshape(boundary, [shape[0], shape[1], 1])
  sflow = tf.reshape(sflow, [shape[0], shape[1], 2])
  vmax = tf.reshape(vmax, [1024])
  boundary = tf.to_float(boundary)
  sflow = tf.to_float(sflow)
  vmax = tf.to_float(vmax)

  return boundary, sflow, vmax

def _generate_image_label_batch(boundary, sflow, vmax, batch_size, shuffle=True):
  num_preprocess_threads = 1
  #Create a queue that shuffles the examples, and then
  #read 'batch_size' images + labels from the example queue.
  boundarys, sflows, vmax = tf.train.shuffle_batch(
    [boundary, sflow, vmax],
    batch_size=batch_size,
    num_threads=num_preprocess_threads,
    capacity=min_queue_examples + 3 * batch_size,
    min_after_dequeue=min_queue_examples)
  return boundarys, sflows, vmax

def flow_inputs(batch_size):
  shape = (128,256)

  tfrecord_filename = glb('./data/*.tfrecords') 
  
  filename_queue = tf.train.string_input_producer(tfrecord_filename) 

  boundary, sflow,vmax = read_data(filename_queue, shape)

  boundarys, sflows, vmax = _generate_image_label_batch(boundary, sflow,vmax, batch_size)

  return boundarys, sflows, vmax

In [3]:
n_batch = 8
learning_rate = 0.0001
keep_prob = 0.7
n_hidden = 128

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32 , [None,128,256,1], name="X")

boundary, sflow, vmax = flow_inputs(n_batch)
X = boundary

v = tf.placeholder(tf.float32 ,[n_batch , 1024], name="v")
# v = tf.to_float(vmax)
v = vmax

print(v)

#Conv1
W1 = tf.Variable(tf.random_normal([16, 16, 1, 128], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 8, 16, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob)

Tensor("shuffle_batch:2", shape=(8, 1024), dtype=float32)


In [4]:
#Cov2
W2 = tf.Variable(tf.random_normal([4,4,128,512], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 4, 4, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, keep_prob)

In [5]:
L2_flat = tf.reshape(L2,[-1,4*4*512])

In [6]:
W3 = tf.get_variable("W", shape=[512 * 4 * 4, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())

In [7]:
b3 = tf.Variable(tf.random_normal([1024]))
L3 = tf.nn.relu(tf.matmul(L2_flat, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

L3 = tf.add(L3 , v)

In [8]:
# L3 = tf.divide(L3,y)
S1, S2 = tf.split(L3, [512, 512], 1)

In [9]:
S1= tf.reshape(S1,[n_batch,1,1,512])
S2= tf.reshape(S2,[n_batch,1,1,512])

In [10]:
# deconv1
W4_1 = tf.Variable(tf.random_normal([8,8,512,512], stddev=0.01))
L4_1 = tf.nn.conv2d_transpose(S1,W4_1,output_shape=[n_batch,8,8,512],strides=[1,8, 8, 1], padding='SAME')
L4_1 = tf.nn.relu(L4_1)
L4_1 = tf.nn.dropout(L4_1, keep_prob)

W4_2 = tf.Variable(tf.random_normal([8,8,512,512], stddev=0.01))
L4_2 = tf.nn.conv2d_transpose(S2,W4_2,output_shape=[n_batch,8,8,512],strides=[1,8, 8, 1], padding='SAME')
L4_2 = tf.nn.relu(L4_2)
L4_2 = tf.nn.dropout(L4_2, keep_prob)

In [11]:
## deconv2
W5_1 = tf.Variable(tf.random_normal([4,8,256,512], stddev=0.01))
L5_1 = tf.nn.conv2d_transpose(L4_1,W5_1,output_shape=[n_batch,32,64,256],strides=[1, 4, 8, 1], padding='SAME')
L5_1 = tf.nn.relu(L5_1)
L5_1 = tf.nn.dropout(L5_1, keep_prob)

W5_2 = tf.Variable(tf.random_normal([4,8,256,512], stddev=0.01))
L5_2 = tf.nn.conv2d_transpose(L4_2,W5_2,output_shape=[n_batch,32,64,256],strides=[1,4, 8, 1], padding='SAME')
L5_2 = tf.nn.relu(L5_2)
L5_2 = tf.nn.dropout(L5_2, keep_prob)

In [12]:
# deconv3
W6_1 = tf.Variable(tf.random_normal([2,2,32,256], stddev=0.01))
L6_1 = tf.nn.conv2d_transpose(L5_1,W6_1,output_shape=[n_batch,64,128,32],strides=[1,2, 2, 1], padding='SAME')
L6_1 = tf.nn.relu(L6_1)
L6_1 = tf.nn.dropout(L6_1, keep_prob)

W6_2 = tf.Variable(tf.random_normal([2,2,32,256], stddev=0.01))
L6_2 = tf.nn.conv2d_transpose(L5_2,W6_2,output_shape=[n_batch,64,128,32],strides=[1,2, 2, 1], padding='SAME')
L6_2 = tf.nn.relu(L6_2)
L6_2 = tf.nn.dropout(L6_2, keep_prob)

In [13]:
# deconv4
W7_1 = tf.Variable(tf.random_normal([2,2,1,32], stddev=0.01))
L7_1 = tf.nn.conv2d_transpose(L6_1,W7_1,output_shape=[n_batch,128,256,1],strides=[1,2,2, 1], padding='SAME')
L7_1 = tf.nn.dropout(L7_1, keep_prob)

W7_2 = tf.Variable(tf.random_normal([2,2,1,32], stddev=0.01))
L7_2 = tf.nn.conv2d_transpose(L6_2,W7_2,output_shape=[n_batch,128,256,1],strides=[1,2, 2, 1], padding='SAME')
L7_2 = tf.nn.dropout(L7_2, keep_prob)

In [14]:
sflow_p = tf.stack([L7_1 , L7_2] , axis=3)
sflow_p = tf.reshape(sflow_p , [n_batch,128,256,2])
# loss = tf.reduce_mean(tf.square(sflow_p - sflow))
loss = tf.nn.l2_loss(sflow_p - sflow)

In [15]:
#loss 
total_loss = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [16]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model_save')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [17]:
tf.train.start_queue_runners(sess=sess)

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 123145580560384)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145584766976)>]

In [18]:
# v = sess.run([v],feed_dict={})
# print(v)

In [20]:
steps = 3000

for epoch in range(steps):    
  total_cost = 0
  _, cost_val = sess.run([total_loss, loss],feed_dict={})
  total_cost += cost_val

  print('Epoch:', '%04d' % (epoch + 1),
        'Avg. cost =', '{:f}'.format(total_cost))
print("최적화 완료") 
saver.save(sess, "./model_save/model.ckpt")

('Epoch:', '0001', 'Avg. cost =', '262.002350')
('Epoch:', '0002', 'Avg. cost =', '222.503235')
('Epoch:', '0003', 'Avg. cost =', '221.210281')
('Epoch:', '0004', 'Avg. cost =', '209.584137')
('Epoch:', '0005', 'Avg. cost =', '198.421478')
('Epoch:', '0006', 'Avg. cost =', '352.496948')
('Epoch:', '0007', 'Avg. cost =', '257.683807')
('Epoch:', '0008', 'Avg. cost =', '224.459259')
('Epoch:', '0009', 'Avg. cost =', '316.796265')
('Epoch:', '0010', 'Avg. cost =', '238.992416')
('Epoch:', '0011', 'Avg. cost =', '114.644539')
('Epoch:', '0012', 'Avg. cost =', '340.642212')
('Epoch:', '0013', 'Avg. cost =', '362.963104')
('Epoch:', '0014', 'Avg. cost =', '156.860443')
('Epoch:', '0015', 'Avg. cost =', '224.454468')
('Epoch:', '0016', 'Avg. cost =', '175.211838')
('Epoch:', '0017', 'Avg. cost =', '338.736572')
('Epoch:', '0018', 'Avg. cost =', '251.358658')
('Epoch:', '0019', 'Avg. cost =', '257.036285')
('Epoch:', '0020', 'Avg. cost =', '149.503464')
('Epoch:', '0021', 'Avg. cost =', '377.6

('Epoch:', '0172', 'Avg. cost =', '355.137268')
('Epoch:', '0173', 'Avg. cost =', '342.259155')
('Epoch:', '0174', 'Avg. cost =', '330.304993')
('Epoch:', '0175', 'Avg. cost =', '74.608780')
('Epoch:', '0176', 'Avg. cost =', '249.329330')
('Epoch:', '0177', 'Avg. cost =', '204.266571')
('Epoch:', '0178', 'Avg. cost =', '148.429001')
('Epoch:', '0179', 'Avg. cost =', '287.817780')
('Epoch:', '0180', 'Avg. cost =', '286.985382')
('Epoch:', '0181', 'Avg. cost =', '116.544785')
('Epoch:', '0182', 'Avg. cost =', '246.187286')
('Epoch:', '0183', 'Avg. cost =', '228.684296')
('Epoch:', '0184', 'Avg. cost =', '210.969482')
('Epoch:', '0185', 'Avg. cost =', '285.893372')
('Epoch:', '0186', 'Avg. cost =', '260.626312')
('Epoch:', '0187', 'Avg. cost =', '413.315094')
('Epoch:', '0188', 'Avg. cost =', '198.409882')
('Epoch:', '0189', 'Avg. cost =', '147.924377')
('Epoch:', '0190', 'Avg. cost =', '271.228088')
('Epoch:', '0191', 'Avg. cost =', '356.988953')
('Epoch:', '0192', 'Avg. cost =', '184.58

('Epoch:', '0343', 'Avg. cost =', '301.057129')
('Epoch:', '0344', 'Avg. cost =', '146.840729')
('Epoch:', '0345', 'Avg. cost =', '194.360992')
('Epoch:', '0346', 'Avg. cost =', '207.754425')
('Epoch:', '0347', 'Avg. cost =', '126.350739')
('Epoch:', '0348', 'Avg. cost =', '143.666519')
('Epoch:', '0349', 'Avg. cost =', '234.071594')
('Epoch:', '0350', 'Avg. cost =', '290.387177')
('Epoch:', '0351', 'Avg. cost =', '277.144592')
('Epoch:', '0352', 'Avg. cost =', '149.486588')
('Epoch:', '0353', 'Avg. cost =', '127.405540')
('Epoch:', '0354', 'Avg. cost =', '66.379608')
('Epoch:', '0355', 'Avg. cost =', '139.101990')
('Epoch:', '0356', 'Avg. cost =', '172.648376')
('Epoch:', '0357', 'Avg. cost =', '200.428406')
('Epoch:', '0358', 'Avg. cost =', '255.366730')
('Epoch:', '0359', 'Avg. cost =', '185.146332')
('Epoch:', '0360', 'Avg. cost =', '157.103912')
('Epoch:', '0361', 'Avg. cost =', '182.910172')
('Epoch:', '0362', 'Avg. cost =', '202.550964')
('Epoch:', '0363', 'Avg. cost =', '262.74

('Epoch:', '0514', 'Avg. cost =', '87.229309')
('Epoch:', '0515', 'Avg. cost =', '184.839813')
('Epoch:', '0516', 'Avg. cost =', '124.178619')
('Epoch:', '0517', 'Avg. cost =', '232.607071')
('Epoch:', '0518', 'Avg. cost =', '124.256996')
('Epoch:', '0519', 'Avg. cost =', '318.655701')
('Epoch:', '0520', 'Avg. cost =', '179.717590')
('Epoch:', '0521', 'Avg. cost =', '67.416862')
('Epoch:', '0522', 'Avg. cost =', '167.320084')
('Epoch:', '0523', 'Avg. cost =', '163.475403')
('Epoch:', '0524', 'Avg. cost =', '156.930008')
('Epoch:', '0525', 'Avg. cost =', '174.206879')
('Epoch:', '0526', 'Avg. cost =', '271.532104')
('Epoch:', '0527', 'Avg. cost =', '264.447052')
('Epoch:', '0528', 'Avg. cost =', '223.515076')
('Epoch:', '0529', 'Avg. cost =', '257.669250')
('Epoch:', '0530', 'Avg. cost =', '141.101822')
('Epoch:', '0531', 'Avg. cost =', '134.307022')
('Epoch:', '0532', 'Avg. cost =', '131.117432')
('Epoch:', '0533', 'Avg. cost =', '262.274139')
('Epoch:', '0534', 'Avg. cost =', '152.036

('Epoch:', '0685', 'Avg. cost =', '233.462494')
('Epoch:', '0686', 'Avg. cost =', '214.571075')
('Epoch:', '0687', 'Avg. cost =', '222.633942')
('Epoch:', '0688', 'Avg. cost =', '229.674591')
('Epoch:', '0689', 'Avg. cost =', '160.817474')
('Epoch:', '0690', 'Avg. cost =', '204.334930')
('Epoch:', '0691', 'Avg. cost =', '123.744720')
('Epoch:', '0692', 'Avg. cost =', '201.868225')
('Epoch:', '0693', 'Avg. cost =', '166.141449')
('Epoch:', '0694', 'Avg. cost =', '246.529053')
('Epoch:', '0695', 'Avg. cost =', '219.660858')
('Epoch:', '0696', 'Avg. cost =', '267.445892')
('Epoch:', '0697', 'Avg. cost =', '151.748810')
('Epoch:', '0698', 'Avg. cost =', '276.010315')
('Epoch:', '0699', 'Avg. cost =', '154.113190')
('Epoch:', '0700', 'Avg. cost =', '183.803726')
('Epoch:', '0701', 'Avg. cost =', '176.403015')
('Epoch:', '0702', 'Avg. cost =', '107.569183')
('Epoch:', '0703', 'Avg. cost =', '192.264343')
('Epoch:', '0704', 'Avg. cost =', '150.330841')
('Epoch:', '0705', 'Avg. cost =', '226.3

...

('Epoch:', '2400', 'Avg. cost =', '252.817490')
('Epoch:', '2401', 'Avg. cost =', '166.176819')
('Epoch:', '2402', 'Avg. cost =', '292.384979')
('Epoch:', '2403', 'Avg. cost =', '208.664200')
('Epoch:', '2404', 'Avg. cost =', '21.204655')
('Epoch:', '2405', 'Avg. cost =', '221.616440')
('Epoch:', '2406', 'Avg. cost =', '147.083252')
('Epoch:', '2407', 'Avg. cost =', '183.511627')
('Epoch:', '2408', 'Avg. cost =', '164.337723')
('Epoch:', '2409', 'Avg. cost =', '169.476196')
('Epoch:', '2410', 'Avg. cost =', '67.948357')
('Epoch:', '2411', 'Avg. cost =', '149.074432')
('Epoch:', '2412', 'Avg. cost =', '261.027252')
('Epoch:', '2413', 'Avg. cost =', '229.072433')
('Epoch:', '2414', 'Avg. cost =', '245.620209')
('Epoch:', '2415', 'Avg. cost =', '95.480644')
('Epoch:', '2416', 'Avg. cost =', '102.628204')
('Epoch:', '2417', 'Avg. cost =', '242.282013')
('Epoch:', '2418', 'Avg. cost =', '118.919174')
('Epoch:', '2419', 'Avg. cost =', '127.757416')
('Epoch:', '2420', 'Avg. cost =', '96.92241

('Epoch:', '2571', 'Avg. cost =', '288.692261')
('Epoch:', '2572', 'Avg. cost =', '215.885559')
('Epoch:', '2573', 'Avg. cost =', '194.662888')
('Epoch:', '2574', 'Avg. cost =', '47.801830')
('Epoch:', '2575', 'Avg. cost =', '110.514450')
('Epoch:', '2576', 'Avg. cost =', '151.927582')
('Epoch:', '2577', 'Avg. cost =', '92.510315')
('Epoch:', '2578', 'Avg. cost =', '157.745117')
('Epoch:', '2579', 'Avg. cost =', '101.404556')
('Epoch:', '2580', 'Avg. cost =', '201.177658')
('Epoch:', '2581', 'Avg. cost =', '170.628967')
('Epoch:', '2582', 'Avg. cost =', '191.429367')
('Epoch:', '2583', 'Avg. cost =', '133.314529')
('Epoch:', '2584', 'Avg. cost =', '178.445755')
('Epoch:', '2585', 'Avg. cost =', '141.863022')
('Epoch:', '2586', 'Avg. cost =', '150.259644')
('Epoch:', '2587', 'Avg. cost =', '241.042099')
('Epoch:', '2588', 'Avg. cost =', '232.523773')
('Epoch:', '2589', 'Avg. cost =', '247.795685')
('Epoch:', '2590', 'Avg. cost =', '176.580215')
('Epoch:', '2591', 'Avg. cost =', '244.327

('Epoch:', '2743', 'Avg. cost =', '222.960724')
('Epoch:', '2744', 'Avg. cost =', '235.134323')
('Epoch:', '2745', 'Avg. cost =', '80.698807')
('Epoch:', '2746', 'Avg. cost =', '125.760887')
('Epoch:', '2747', 'Avg. cost =', '187.774841')
('Epoch:', '2748', 'Avg. cost =', '97.759735')
('Epoch:', '2749', 'Avg. cost =', '218.821121')
('Epoch:', '2750', 'Avg. cost =', '97.239273')
('Epoch:', '2751', 'Avg. cost =', '137.939117')
('Epoch:', '2752', 'Avg. cost =', '182.234863')
('Epoch:', '2753', 'Avg. cost =', '226.902222')
('Epoch:', '2754', 'Avg. cost =', '136.999100')
('Epoch:', '2755', 'Avg. cost =', '218.528015')
('Epoch:', '2756', 'Avg. cost =', '183.730133')
('Epoch:', '2757', 'Avg. cost =', '111.399979')
('Epoch:', '2758', 'Avg. cost =', '189.138031')
('Epoch:', '2759', 'Avg. cost =', '163.080994')
('Epoch:', '2760', 'Avg. cost =', '118.720764')
('Epoch:', '2761', 'Avg. cost =', '106.177132')
('Epoch:', '2762', 'Avg. cost =', '251.297272')
('Epoch:', '2763', 'Avg. cost =', '211.9594

('Epoch:', '2915', 'Avg. cost =', '242.576080')
('Epoch:', '2916', 'Avg. cost =', '150.321625')
('Epoch:', '2917', 'Avg. cost =', '199.441345')
('Epoch:', '2918', 'Avg. cost =', '238.442917')
('Epoch:', '2919', 'Avg. cost =', '157.946564')
('Epoch:', '2920', 'Avg. cost =', '250.351929')
('Epoch:', '2921', 'Avg. cost =', '181.627884')
('Epoch:', '2922', 'Avg. cost =', '134.751129')
('Epoch:', '2923', 'Avg. cost =', '257.889709')
('Epoch:', '2924', 'Avg. cost =', '181.067505')
('Epoch:', '2925', 'Avg. cost =', '128.660919')
('Epoch:', '2926', 'Avg. cost =', '99.741196')
('Epoch:', '2927', 'Avg. cost =', '198.759216')
('Epoch:', '2928', 'Avg. cost =', '134.322220')
('Epoch:', '2929', 'Avg. cost =', '240.314575')
('Epoch:', '2930', 'Avg. cost =', '145.027832')
('Epoch:', '2931', 'Avg. cost =', '205.760773')
('Epoch:', '2932', 'Avg. cost =', '277.130005')
('Epoch:', '2933', 'Avg. cost =', '169.653015')
('Epoch:', '2934', 'Avg. cost =', '207.042847')
('Epoch:', '2935', 'Avg. cost =', '128.20

'./model_save/model.ckpt'